## Imports

In [1]:
import numpy as np
import pandas as pd
import math
from functools import total_ordering

# Code

In [2]:
class number:
    def __init__(self, digits):
        if(type(digits) == int):
            r = ''
            n = digits
            while n != 0:
                a = n % 2
                r = str(int(a))+r
                n = (n - (n % 2))/-2
            digits = r    
        self.digits = digits.lstrip('0')
        if(len(self.digits)==0):
            self.digits = '0'
        self.len = len(self.digits)
        
    def __repr__(self):
        return 'Antibinary integer: ' + self.digits
    
    def __str__(self):
        return self.digits
    
    def __add__(self, b):
        if(type(b) != number):
            b = number(b)
        return self.add(b)
    
    def __mul__(self, b):
        if(type(b) != number):
            b = number(b)
        return self.mult(b)
    
    def __sub__(self, b):
        if(type(b) != number):
            b = number(b)
        return self.subtract(b)
    
    def __truediv__(self, b):
        if(type(b) != number):
            b = number(b)
        return self.mult(b.recip())
    
    def __pow__(self, b):
        r =  abfloat(1)
        if(b < 0):
            a = self.recip()
        elif (b==0):
            return r
        else:
            a = self
        for i in range(abs(b)):
            r *= a
        return r
        
    def to_str(self):
        return self.digits
    
    def add(self, b):
        a = self
        c = ''.zfill(2 + max(self.len, b.len))
        ad = list(a.digits.zfill(len(c)))
        bd = list(b.digits.zfill(len(c)))
        o = list(c[:])
        c = list(c)
        for i in range(len(c)-1,-1,-1):
            o[i] = '0' if ((ad[i] == bd[i]) == (c[i] == '0')) else '1'
            if ((c[i] == '1' and (bd[i] != ad[i])) or (bd[i] == '1' and ad[i] == '1')):
                if(c[i-1] == '1'):
                    c[i-1] = '0'
                else:
                    c[i-1], c[i-2] = '1', '1'
        return number(''.join(o).lstrip('0'))
    
    def subtract(self, b):
        return self.add(number('11').mult(b))
    
    def mult(self, b):
        a = self
        c = number(''.zfill(2 + a.len + b.len))
        for i in range(b.len-1, -1, -1):
            if list(b.digits)[i] == '1':
                x = number(a.to_str()[::-1].zfill(a.len+b.len-i-1)[::-1])
                c = c.add(x)
        return number(c.digits)
    
    def to_decimal(self):
        r = 0
        for i in range(self.len):
            r = r + int(self.digits[self.len-1-i:self.len-i])*(-2)**i
        return r

## Integer Code

In [3]:
@total_ordering
class abfloat:
    def __init__(self, k = 0):
        if(type(k) == tuple):
            self.exp = number(k[0]).to_str()
            self.mant = k[1][0:53]
        elif(type(k) == str):
            if(('.') in k):
                n = k.index('.')
                k = k.replace('.','')
                l = len(k)-n
                self.exp = number(number(abfloat(number(k)).exp).to_decimal() - l).digits
                self.mant = abfloat(number(k)).mant
            else:
                self.exp = abfloat(number(k)).exp
                self.mant = abfloat(number(k)).mant
        else:
            if(type(k) == number):
                k = k.to_decimal()
            if(type(k) == int or type(k) == float):
                a, b = math.frexp(float(k))
                b -= 1
                a *= 2*(-1)**(b%2)
                if(a > 4/3):
                    a /= 4
                    b += 2
                if(a <= -1):
                    a /= -2
                    b += 1
                a = number((int) (a*(-2)**53)).digits
                self.exp = number(b).digits
                self.mant = a[1:54]

    def __repr__(self):
        return 'Antibinary float: [' + self.exp + ',' + self.mant +']'
    
    def __str__(self):
        return self.print_ab()
    
    def __add__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        return self.add(b)
    
    def __mul__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        return self.mult(b)
    
    def __sub__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        return self.subtract(b)
    
    def __truediv__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        return self.mult(b.recip())
    
    def __pow__(self, b):
        if(type(b) == number):
            b = number(b).to_decimal()
        if(type(b) == int):
            r =  abfloat(1)
            if(b < 0):
                a = self.recip()
            elif (b==0):
                return r
            else:
                a = self
            for i in range(abs(b)):
                r *= a
            return r
        else:
            return (abs(self).ln()*b).ett()
    
    def __eq__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        return (number(((self-b)/self).exp).to_decimal() < -10)
    
    def __lt__(self, b):
        if(type(b) != abfloat):
            b = abfloat(b)
        if (self==b):
            return False
        return '1' == ((self-b).exp)[-1]
    
    def __abs__(self):
        if(self < 0):
            return self*-1
        else:
            return self
        
    def print_ab(self, d = 30):
        sms = ('1'+self.mant)[0:d].rstrip('0')
        exp = number(self.exp).to_decimal() + 1
        if(abs(exp)>10):
            return ('1.'+sms+' b '+self.exp)
        else:
            if(exp >= 0):
                if(len(sms) <= exp):
                    return (sms + ''.join(['0']*(max(0, exp-len(sms)))))
                else:
                    return (sms[0:exp]+'.'+sms[exp:min(len(sms), exp+5)])
            else:
                return ('.'+(''.join(['0']*(-1*exp)))+sms)
                
    def to_decimal(self):
        n = '1'+self.mant
        return (number(n).to_decimal())*(-2)**(-53+number(self.exp).to_decimal())
    
    def add(self, b):
        if(type(b) == number):
            return self.add(abfloat(b))
        ae = number(self.exp).to_decimal()
        be = number(b.exp).to_decimal()
        if(ae<be):
            return b.add(self)
        am = '1'+self.mant+(''.join(['0']*(ae-be)))
        bm = '1'+b.mant
        mant = (number(am).add(number(bm))).digits
        e = be + len(mant) - 54
        mant = mant[1:max(len(mant)-1, 54)]
        mant = mant + ''.join(['0']*(53 - len(mant)))
        return abfloat((e, mant))
    
    def subtract(self, b):
        return self.add(b.mult(number('11')))
    
    def mult(self, b):
        if(type(b) == number):
            return self.mult(abfloat(b))
        ae = number(self.exp).to_decimal()
        be = number(b.exp).to_decimal()
        am = '1'+self.mant
        bm = '1'+b.mant
        mant = number(am).mult(number(bm)).digits
        e = ae + be + len(mant) - 107
        mant = mant[1:min(len(mant)-1, 54)]
        mant = mant + ''.join(['0']*(53 - len(mant)))
        return abfloat((e, mant))
    
    def recip(self):
        a = abfloat((0,self.mant))
        d = a-(abfloat((0, ''.join(['1']*54))))
        d = (d*d*abfloat('111.1'))-(d*abfloat('111.1'))+abfloat('110.111')
        for i in range(10):
            d *= (a*d*(-1) + 2)
        return d*((-2)**(number('11')*number(self.exp)).to_decimal())

    def ett(self):
        a = abfloat(self.to_decimal())
        r = abfloat(0)
        b = abfloat(2)
        x = (b-1)/(b+1)
        for i in range(40):
            r += (x**(2*i+1))*1/(2*i+1)
        r *= 2
        e = 1
        while(a < 0):
            a += r*2
            e /= 4
        while(a > 3):
            a -= r*2
            e *= 4
        r = abfloat(0)
        for i in range(60):
            r += (a**i)*(1/math.factorial(i))
        return r*e
    
    def ln(self):
        r = abfloat(0)
        a = abfloat(2)
        x = (a-1)/(a+1)
        for i in range(40):
            r += (x**(2*i+1))*1/(2*i+1)
        r *= abfloat(self.exp)*2
        r2 = abfloat(0)
        a = (abfloat((0,self.mant)) - 1)*-1
        b = abfloat(1)
        for i in range(1,60):
            b *= a
            r2 += b*(1/i)
        return r - r2

# Demo

In [6]:
number(77).to_decimal()

v = number(-307)
c = number(273)
v.add(c).digits

v.add(c)

for k in [v, c, v.add(c), v.mult(c)]:
    print(k.digits)
    print(k.to_decimal())

1111011101
-307
100010001
273
100010
-34
111100100111101101
-83811


In [5]:
qi = 6.951
ri = -170.5555
q = abfloat(qi)
r = abfloat(ri)
print(qi, ri)
print(q,r)
print((q+r).to_decimal(), qi+ri)
print((q*r).to_decimal(), qi*ri)
print((q-r).to_decimal(), qi-ri)
print((q/r).to_decimal(), qi/ri)
print((r**4).to_decimal(), ri**4)
print(((q-7)*(r+5))<(q**3), ((qi-7)*(ri+5))<(qi**3))
print((r**q).to_decimal(), abs(ri)**qi)

6.951 -170.5555
11011.00110 10101010.10110
-163.60449999999997 -163.6045
-1185.5312804999996 -1185.5312804999999
177.5065 177.5065
-0.04075506213519939 -0.040755062135199395
846180310.4736751 846180310.4736753
True True
3263613204849227.5 3263613204849409.0
